In [ ]:
from dataset import CLDataset

In [ ]:
from itertools import islice
sub_dict = {}
with CLDataset.get_reader('dataset/Task2.csv') as reader:
    for row in reader:
        if row['iSub'] not in sub_dict:
            sub_dict[row['iSub']] = {}
            sub_dict[row['iSub']]['index'] = row['iSub']
            sub_dict[row['iSub']]['session'] = row['iSession']
            sub_dict[row['iSub']]['version'] = row['version']
            sub_dict[row['iSub']]['condition'] = row['condition']
            sub_dict[row['iSub']]['exp'] = []
        exp_list = sub_dict[row['iSub']]['exp']
        trial = row['iTrial']
        feature = [row['feature1'], row['feature2'], row['feature3'], row['feature4']]
        text = row['text']        
        category = row['category']
        choice = row['choice']
        feadback = row['feedback']
        exp_list.append({'trial': trial, 'feature': feature, 'text': text, 'category': category, 'choice': choice, 'feedback': feadback})


In [ ]:
from dataset import CLDataset
sub_dict = CLDataset.get_sub_dict('dataset/Task2.csv')
sub_dict = {k: v for k, v in sub_dict.items() if v['version'] != '2'}

In [ ]:
import csv
f = open('dataset/Task2.csv', encoding="GBK", errors="ignore", mode="r", newline="")
reader = csv.DictReader(f)
reader.fieldnames

In [ ]:
import torchaudio
import numpy as np
waveform, sample_rate = torchaudio.load('Recording/9_1/128.wav')
waveform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)(waveform)

audio = waveform.squeeze().numpy()

import whisper
# 加载Whisper预训练模型
model = whisper.load_model("turbo")

result = model.transcribe(audio)

In [ ]:
import torchaudio
import numpy as np


import speech_recognition as sr
r = sr.Recognizer()
with sr.AudioFile('Recording/9_1/128.wav') as source:
    audio = r.record(source)
result = r.recognize_google_cloud(audio)
result



In [ ]:
print(result['text'])

In [ ]:
import glob
recording_dir = glob.glob('Recording/*')
recording_dir.sort(key=lambda x: (int(x.split('/')[-1].split('_')[0]), int(x.split('/')[-1].split('_')[1].split('.')[0])))
recording_dir

In [ ]:
import glob
recording_dir = glob.glob('Recording/*')
recording_dir = [x for x in recording_dir if x.split('/')[-1].split('_')[0] in ['8', '13', '15', '19', '20', '21']]
recording_dir.sort(key=lambda x: (int(x.split('/')[-1].split('_')[0]), int(x.split('/')[-1].split('_')[1].split('.')[0])))
recording_dir

In [ ]:
import os
remote_base_dir = 'gpu6/categoryLearning/'
for dir in recording_dir:
    remote_dir = remote_base_dir+dir
    files = glob.glob(dir+'/*')
    print(files)
    print(remote_dir)
    break

## 聚类

In [ ]:
import pandas as pd
df = pd.read_csv('dataset/Task2.csv', encoding='GBK')
texts = df['text'].tolist()

In [ ]:
import pandas as pd
df_xlsx = pd.read_excel('dataset/1_1.xlsx')
texts = df_xlsx['text'].tolist()

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, BertModel
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")

model = BertModel.from_pretrained("bert-base-chinese")


# tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
# model = BertModel.from_pretrained("google-bert/bert-base-uncased")

inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=128)
outputs = model(**inputs)

vectors = outputs.pooler_output

In [ ]:
from sklearn.cluster import DBSCAN

# 使用DBSCAN进行聚类
dbscan = DBSCAN(eps=2, min_samples=2)
dbscan_clusters = dbscan.fit_predict(vectors.detach().numpy())


In [ ]:
df_clusters = pd.DataFrame({'cluster': dbscan_clusters, 'text': texts})
df_clusters.head()

In [ ]:
df_clusters[df_clusters['cluster'] == 0]

In [ ]:
from sklearn.cluster import KMeans
KMeans_model = KMeans(n_clusters=2, random_state=0)
KMeans_clusters = KMeans_model.fit_predict(vectors.detach().numpy()[65:96,:])

In [ ]:
df_clusters = pd.DataFrame({'cluster': KMeans_clusters, 'text': texts[65:96]})

## rnn

In [ ]:
import pandas as pd
df_xlsx = pd.read_excel('dataset/1_1.xlsx')
texts = df_xlsx['text'].tolist()


In [ ]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

import torch
from torch.utils.data import DataLoader, Dataset

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=128)

In [ ]:
input_ids = [inputs['input_ids'][i:i+10] for i in range(0, len(inputs['input_ids'])-10)]

In [ ]:
class TextDataset(Dataset):
    def __init__(self, input_ids):
        self.input_ids = input_ids

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx]

dataset = TextDataset(input_ids)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

In [ ]:
for x in dataloader:

    print(x.shape)




In [ ]:
x[0][0]

In [ ]:
from torch import nn
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = x.to(torch.float32)
        out, _ = self.rnn(x)
        out = self.fc(out)
        return out



In [ ]:
model = RNNModel(58, 128, 2, 58)


In [ ]:
torch.argmax(x, dim=-1)[:,1:].shape

In [ ]:
out = model(x[:,:-1])

In [ ]:
nn.functional.cross_entropy(out.reshape(-1, 58), torch.argmax(x, dim=-1)[:,1:].reshape(-1), ignore_index=0)



In [ ]:
model.rnn

In [12]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

import pandas as pd
from tqdm.auto import tqdm

sub_train_id = [2 , 3]

df = pd.read_csv('/workspace/category-learning/recording_finish.csv', keep_default_na=False)
texts = []
for sub_id in tqdm(sub_train_id, desc='Loading texts'):
    text = df[df['iSub'] == sub_id]['text'].tolist()
    texts.append(text)
from transformers import AutoTokenizer
import torch
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")


from transformers import AutoTokenizer, BertModel
import torch
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
toks = []
for text in texts:
    tok = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    toks.append(tok)
# max_length = max(tok['input_ids'].shape[1] for tok in toks)

# padded_toks = []
# for tok in toks:
#     input_ids = torch.nn.functional.pad(tok['input_ids'], (0, max_length - tok['input_ids'].shape[1]), value=tokenizer.pad_token_id)
#     attention_mask = torch.nn.functional.pad(tok['attention_mask'], (0, max_length - tok['attention_mask'].shape[1]), value=0)
#     padded_toks.append({'input_ids': input_ids, 'attention_mask': attention_mask})

# model = BertModel.from_pretrained("bert-base-chinese")
# model.eval()
# for tok in padded_toks:
#     inputs = {k: v.unsqueeze(0) for k, v in tok.items()}
#     outputs = model(**inputs)

#     vectors = outputs.pooler_output

Loading texts: 100%|██████████| 2/2 [00:00<00:00, 620.09it/s]

In [18]:
model = BertModel.from_pretrained("bert-base-chinese")
all_text_vector = model(**toks[0]).pooler_output

In [1]:
import torch
all_tok = torch.load('/workspace/category-learning/all_tok.pt')
all_choice = torch.load('/workspace/category-learning/all_choice.pt')
all_feedback = torch.load('/workspace/category-learning/all_feedback.pt')
sub_exp_start = torch.load('/workspace/category-learning/sub_exp_start.pt')

/tmp/ipykernel_150201/4117357251.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  all_tok = torch.load('/workspace/category-learning/all_tok.pt')
/tmp/ipykernel_150201/41

In [2]:
from torch.utils.data import Dataset
class CategoryDataset(Dataset):
    def __init__(self, text, choice, feedback, sub_exp_start):
        self.all_data = []
        for i in range(len(sub_exp_start)-1):
            start = sub_exp_start[i]
            end = sub_exp_start[i+1]
            for j in range(start, end-11):
                self.all_data.append((torch.stack([text[k] for k in range(j, j+10)]),
                                      torch.stack([choice[k] for k in range(j, j+10)]),
                                      torch.stack([feedback[k] for k in range(j, j+10)])))
                                      


    def __len__(self):
        return len(self.all_data)

    def __getitem__(self, idx):
        return self.all_data[idx]
    
dataset = CategoryDataset(all_tok, all_choice, all_feedback, sub_exp_start)

In [3]:
from rnn.core import CategoryRNN
from transformers import AutoTokenizer
import torch
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
model = CategoryRNN(len(tokenizer.get_vocab()), 768, 4, 2)

/root/miniconda3/envs/category/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model.cal_loss(dataset[0])

tensor(1.4727, grad_fn=<AddBackward0>)

In [15]:
from torch.utils.data import DataLoader
def collate_fn(batch):
    return batch
for data in DataLoader(dataset, batch_size=1, shuffle=True, collate_fn=collate_fn):
    print(data[0])
    break

(tensor([[ 101, 7531, 4764, 5556, 2094, 4764,  102,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0],
        [ 101, 1928, 7270, 5597, 4764,  102,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0],
        [ 101, 5556, 2094, 7270, 1928, 4764,  102,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0],
        [ 101, 1928, 4764, 5556, 2094, 4764,  102,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0],
        [ 101, 7531, 4764, 5556, 2094, 4764,  102,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
     

## 繁体转简体

In [1]:
csv_file = '/workspace/category-learning/recording-1217.csv'
import pandas as pd
df = pd.read_csv(csv_file, encoding='utf', sep=',')

In [2]:
import opencc
from tqdm.auto import tqdm
converter = opencc.OpenCC('t2s')
for i in tqdm(range(len(df)), desc='Converting text'):
    text = df.loc[i, 'text']
    if pd.isna(text):
        continue
    text = text.replace(' ', '').strip('"')
    df.loc[i, 'text'] = converter.convert(df.loc[i, 'text'])

/root/miniconda3/envs/category/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Converting text:   0%|          | 0/313 [00:00<?, ?it/s]

Converting text: 100%|██████████| 313/313 [00:00<00:00, 3320.61it/s]


In [3]:
df.to_csv('/workspace/category-learning/recording-1217.csv', index=False, encoding='utf')

In [5]:
import pandas as pd
df = pd.read_csv('../recording-1217.csv', encoding='utf')
for i in range(len(df)):
    text = str(df.loc[i, 'text']).rstrip('"').lstrip('"')
    df.loc[i, 'text'] = f'{text}'
    # if '"' not in text:
    #     df.loc[i, 'text'] = f'"{text}"'
df.to_csv('/workspace/category-learning/recording-1217.csv', index=False, encoding='utf')

In [11]:
for sub_id, group in df.groupby('iSub'):
    if sub_id == 17:
        continue
    group.to_csv(f'../Task2_{sub_id}_aud.csv', index=False, encoding='utf-8')

## 写入文件

In [1]:
import pandas as pd
import glob
aud_files = glob.glob('../dataset/aud/Task2_*_aud.csv')
aud_files = sorted(aud_files, key=lambda x: int(x.split('_')[1]))
df = pd.DataFrame()
for aud_file in aud_files:
    try:
        df_ = pd.read_csv(aud_file, encoding='utf-8')
    except:
        df_ = pd.read_csv(aud_file, encoding='GBK')
    df = pd.concat([df, df_])


In [3]:
df['iSub'].unique()

array([ 1,  2,  3,  7,  8,  9, 12, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24,
       25])

In [15]:
df.to_csv('../dataset/Task2_aud.csv', index=False, encoding='utf-8')